In [1]:
import typing

# Сбор данных

In [ ]:
%pip install pandas
%pip install bs4
%pip install requests 
%pip install fake_useragent

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent
from tqdm.auto import tqdm
import pandas as pd

In [3]:
ua = UserAgent()
session = requests.session()

In [5]:
def get_text(block: typing.Dict[str, str]) -> typing.Dict[str, str]:
    """
    Преобразует данные блока информации.

    Args:
        block (Dict[str, str]): Словарь с данными блока информации.

    Returns:
        Dict[str, str]: Преобразованный словарь с данными.
    """
    
     # Обход всех ключей и значений в словаре block
    for k, v in block.items():
        try:
            # Проверка ключа на принадлежность к определенным категориям
            if k in ['URL', 'название источника']:
                pass
            elif k == 'текст статьи':
                # Обработка текста статьи: объединение и очистка текстовых элементов
                block[k] = ''.join([i.text.strip() for i in v]).replace('\xa0', ' ')
            else:
                # Обработка остальных категорий данных
                block[k] = v.text.strip().replace('\xa0', ' ').replace('| DOCTORPITER', '')
        except Exception:
            # Обработка исключения при возникновении ошибки
            block[k] = ''
      
    # Словарь месяцев для преобразования даты
    months = {'января': '01',
              'февраля': '02',
              'марта': '03', 
              'апреля': '04',
              'мая': '05', 
              'июня': '06',
              'июля': '07', 
              'августа': '08', 
              'сентября': '09', 
              'октября': '10', 
              'ноября': '11', 
              'декабря': '12'}

    # Преобразование даты в формат "год-месяц-день"
    if block.get('дата'):
        date_parts = block['дата'].split(' ')
        if len(date_parts) == 3:
            day, month, year = date_parts
            if day.isdigit() and month in months and year.isdigit():
                month_num = months[month]
                if int(day) < 10:
                    day = f'0{day}'
                block['дата'] = f'{year}-{month_num}-{day}'

    return block

In [ ]:
def parse_one_article(link: str) -> typing.Dict[str, str]:
    """
    Парсит информацию со страницы статьи.

    Args:
        link (str): Ссылка на страницу статьи.

    Returns:
        Dict[str, str]: Словарь с данными статьи.
    """

    # Создаем пустой словарь для хранения данных статьи
    block = dict()

    # Отправляем GET-запрос к странице статьи
    req = session.get(link, headers={'User-Agent': ua.random})
    page = req.text

    # Парсинг HTML страницы
    soup = BeautifulSoup(page, 'html.parser')

     # Извлекаем данные со страницы и записываем в словарь
    block['URL'] = link
    block['название источника'] = 'Доктор Питер'
    block['дата'] = soup.find('span', {'class': 'ds-article-header-date-and-stats__date'})
    block['автор'] = soup.find('span', {'class': 'ds-article-footer-authors__author'})
    block['название статьи'] = soup.find('title')
    block['текст статьи'] = soup.find_all(
        'div', 
        {'ds-block-text text-style-body-1 ds-article-content__block ds-article-content__block_text'}
        )

    # Вызов функции get_text для обработки данных и возвращение результата
    return get_text(block)

In [7]:
def get_nth_page(section: str, page_number: int) -> list:
    """
    Получает ссылки на новости с указанной страницы раздела сайта https://doctorpiter.ru/
    и возвращает список новостей.

    Args:
        section (str): Раздел сайта, например 'novosti', 'releases', 
        'zdorove', 'obraz-zhizni', 'pravilnoe-pitanie', 'tags/serdce/', 'tags/diagnostika'.
        page_number (int): Номер страницы для извлечения новостей.

    Returns:
        list: Список новостей с указанной страницы раздела.

    """
    
    # Формирование URL для запроса
    url = f'https://doctorpiter.ru/{section}/page-{page_number}/'
    
    # Отправка GET-запроса на указанный URL с помощью сессии
    req = session.get(url, headers={'User-Agent': ua.random})
    
    # Получение текста страницы
    page = req.text
    
    # Создание объекта BeautifulSoup для парсинга HTML
    soup = BeautifulSoup(page, 'html.parser')
    
    # Поиск всех элементов новостей на странице
    links = soup.find_all(('a', {'class': 'announce-inline-tile__content mr-d-2'}))
    
    # Регулярное выражение для поиска ссылок на новости
    rex = r'(?<=href=")(\/(?:novosti|releases|zdorove|obraz-zhizni|pravilnoe-pitanie|tags\/serdce\/|tags\/diagnostika)\/[a-zA-Z0-9-]+\/)'
    
    # Извлечение ссылок на новости и фильтрация
    links = [re.search(rex, str(i))[0] for i in links if re.search(rex, str(i)) is not None]
    links = [i for i in links if 'page-' not in i]
    links = [f'https://doctorpiter.ru/{i}' for i in links]

    news = []
    
    # Парсинг каждой новости по ссылке и добавление в список новостей
    for l in links:
        try: 
            news.append(parse_one_article(l))
        except Exception:
            pass

    return news

In [8]:
def get_n_pages(section: str, n_pages: int) -> list:
    """
    Получает новости из указанного раздела сайта https://doctorpiter.ru/ и возвращает список всех новостей.

    Args:
        section (str): Раздел сайта, например 'novosti', 'releases', 
        'zdorove', 'obraz-zhizni', 'pravilnoe-pitanie', 'tags/serdce/', 'tags/diagnostika'.
        n_pages (int): Количество страниц для извлечения новостей.

    Returns:
        list: Список всех новостей из указанного раздела и страниц.
    """
    
    blocks = []
    
    # Итерация по страницам для получения новостей
    for i in tqdm(range(1, n_pages+1)):
        blocks.extend(get_nth_page(section, i))

    return blocks

In [9]:
def get_data(section: str, n_pages: int) -> list:
    """
    Функция для получения данных из разделов и сохранения их в CSV файл.

    Args:
        section (str): Раздел сайта, например 'novosti', 'releases', 'zdorove', 
        'obraz-zhizni', 'tags/pravilnoe-pitanie', 'tags/serdce', 'tags/diagnostika'.
        n_pages (int): Количество страниц для извлечения новостей.

    Returns:
        sections (list): Список данных из разделов.
    """

    # Получение данных из раздела
    blocks = get_n_pages(section, n_pages)  

    # Сохранение данных
    if section == 'novosti':
        first = pd.DataFrame(blocks)
        first.to_csv('Doctor_Piter.csv', index=False)
    else:
        previous = pd.read_csv('Doctor_Piter.csv')
        new = pd.DataFrame(blocks)
        final = pd.concat([previous, new], ignore_index=True)
        final.to_csv('Doctor_Piter.csv', index=False)

    # Промежуточное сообщение о результатах скачивания
    print(f'Данные из раздела "{section}" успешно скачаны и сохранены.')
    
    return blocks

In [ ]:
data = get_data('novosti', 203)

In [ ]:
data.extend(get_data('/tags/pravilnoe-pitanie', 97))

In [35]:
# Удаление дубликатов
df = pd.read_csv('Doctor_Piter.csv')
df_unique = df.drop_duplicates()
df_unique.to_csv('Doctor_Piter.csv')

In [43]:
def transform_data(name: str) -> str:
    """
    Функция для преобразования данных из CSV файла в строку 
    по заданному формату и записи в текстовый файл.

    Args:
        name (str): Название файла без расширения.

    Returns:
        transformed_data (str): Преобразованные данные в виде строки.
    """

    # Чтение данных из файла 
    data = pd.read_csv(f'{name}.csv')
    data = data.loc[:, 'URL':].values.tolist()
    for d in range(len(data)):
        for w in range(len(data[d])):
            # замена nan на пустую строку
            if isinstance(data[d][w], float):
                data[d][w] = ''

    # Преобразование данных в строку по заданному формату
    data = ['\n'.join(map(str, d)) for d in data]
    text = '\n=====\n'.join(data)


    # Запись данных в текстовый файл
    with open(f'{name}.txt', 'w', encoding='utf-8') as file:
        file.write(text)

    return text

In [56]:
df = pd.read_csv('Doctor_Piter.csv')
texts = df['текст статьи'].tolist()

# Преобразование текста
for i in range(len(texts)):
    text = texts[i]
    if not isinstance(text, float):
        new_text = ''
        for e in range(len(text)-1):
            if text[e] in ['.', '!', '?'] and text[e+1] not in '0123456789,.!?':
                new_text += text[e] + ' '
            elif text[e].islower() and text[e+1].isupper():
                new_text += text[e] + '.\n'
            else:
                new_text += text[e]

        # Добавление последнего символа к новой строке
        new_text += text[-1]

        # Замена исходного текста на преобразованный
        texts[i] = new_text

# Обновление столбца 'текст статьи' с преобразованными текстами
df['текст статьи'] = texts

# Сохранение обновленного датафрейма в CSV файл
df.to_csv('Doctor_Piter.csv', index=False)

In [ ]:
transform_data('Doctor_Piter')

# Обработка данных

In [ ]:
%pip install nltk
%pip install pymystem3
%pip install pandas

In [ ]:
import re
import nltk
from nltk.tokenize import sent_tokenize
from pymystem3 import Mystem
nltk.download('punkt')
import pandas as pd
from tqdm.auto import tqdm


In [50]:
def preprocess_one_text(text: str) -> list: 
    """
    Функция принимает на вход текст в виде строки и 
    возвращает список предложений после предобработки.

    Аргументы:
        text: str - исходный текст для обработки

    Возвращаемое значение:
        list - список предложений после предобработки

    """

    try: 
        # Удаление лишних пробелов и символов
        text = text.replace(', ', ' ').replace(' - ', ' ').replace('\n', ' ')

        # Разделение текста на предложения
        sentences = sent_tokenize(text, language='russian')

        # Удаление ссылок и специальных символов
        sentences = [re.sub(r'http\S+', '', s) for s in sentences]
        sentences = [re.sub(r'[^\w\s]', '', s) for s in sentences]

        # Приведение к нижнему регистру
        sentences = [s.lower() for s in sentences]

        m = Mystem()

        # Лемматизация слов
        sentences = [''.join(
            m.lemmatize(s)).replace('  ', ' ').replace('\n', '') for s in sentences]
        return sentences
    
    except Exception: 
        with open('errors.txt', 'a', encoding='utf-8') as file:
            file.write(f'{text}\n\n')
        return None


In [52]:
def preprocess(name: str) -> list: 
    """
    Функция для предобработки текстов .
    
    Args:
        name (str): Название файла без расширения
        
    Returns:
        list: Список лемматизированных предложений.
    """

    texts = list(pd.read_csv(f'{name}.csv', encoding='utf-8')['текст статьи']) 
    texts = [preprocess_one_text(texts[t]) for t in tqdm(range(len(texts)))]
    
    return texts

In [ ]:
data = preprocess('Doctor_Piter')

In [54]:
with open('lemmatized_text.txt' , 'w', encoding='utf-8') as file:
    file.writelines(f"{i}\n" for i in data)

# Построение графа

In [55]:
from nltk.util import ngrams
from itertools import combinations
from tqdm.auto import tqdm
import random

In [56]:
def count_word_occurrences_in_ngrams(
        ngrams: list) -> typing.Dict[str, typing.Dict[str, int]]:
    """
    Функция для подсчета встречаемости слов в n-граммах.
    
    Args:
        ngrams (list): Список n-грамм.
        
    Returns:
        Dict[str, Dict[str, int]]: Словарь с встречаемостью слов в n-граммах.
    """
    
    word_counts = {}
    all_pairs = []
    
    # Создание всех возможных пар слов из n-грамм
    for n in ngrams: 
        all_pairs.extend(list(combinations(n, 2)))
    
    # Подсчет встречаемости слов в парах
    for pair in tqdm(range(len(all_pairs))):
        if all_pairs[pair][0] not in word_counts:
            word_counts[all_pairs[pair][0]] = {}
            word_counts[all_pairs[pair][0]][all_pairs[pair][1]] = 1
        else:
            if all_pairs[pair][1] not in word_counts[all_pairs[pair][0]]:
                word_counts[all_pairs[pair][0]][all_pairs[pair][1]] = 1
            else:
                word_counts[all_pairs[pair][0]][all_pairs[pair][1]] += 1

        if all_pairs[pair][1] not in word_counts:
            word_counts[all_pairs[pair][1]] = {}
            word_counts[all_pairs[pair][1]][all_pairs[pair][0]] = 1
        else:
            if all_pairs[pair][0] not in word_counts[all_pairs[pair][1]]:
                word_counts[all_pairs[pair][1]][all_pairs[pair][0]] = 1
            else:
                word_counts[all_pairs[pair][1]][all_pairs[pair][0]] += 1

    return word_counts

In [83]:
def create_ngrams(n: int,  k: int = 6869) -> list:
    """
    Функция для создания n-грамм из списка лемматизированных предложений.
    
    Args:
        n (int): Значение n для создания n-грамм.
        k (int): Значение k для отбора k текстов.
        
    Returns:
        list: Список n-грамм.
    """
    
    # Чтение файла и изъятие данных для построения n-грамм
    with open('lemmatized_text.txt', 'r', encoding='utf-8') as file:
        sentences = file.read().replace('[', '').replace(']', '').replace("'", '').split('\n')[:-1]
        # случайная выборка из k текстов
        sentences = random.sample(sentences, k)
        sentences = [s.split(', ') for s in sentences]

    # Создание n-грамм 
    list_of_ngrams = []
    for t in sentences:
        for s in t: 
            list_of_ngrams.extend(list(ngrams(s.split(), n)))

    print(f'Количество {n}-грамм: {len(list_of_ngrams)}')
    return list_of_ngrams

In [ ]:
%pip install networkx
%pip install matplotlib

In [59]:
import networkx as nx
import matplotlib.pyplot as plt

In [62]:
def build_graph(
        data: typing.Dict[str, typing.Dict[str, int]]) -> nx.Graph:
    """
    Функция строит граф на основе данных из словаря.
    
    Parameters:
    data (Dict[str, Dict[str, int]]): Словарь с данными для построения графа.
    
    Returns:
    nx.Graph: Построенный граф.
    """
    # Добавление дуг в граф.
    G = nx.Graph()
    for source_node, target_nodes in tqdm(data.items()):
        for target_node, weight in target_nodes.items():
            G.add_edge(source_node, target_node, weight=weight)

    return G


In [63]:
def draw_graph(G: nx.Graph, n: int) -> nx.Graph:
    """
    Функция визуализирует граф G с помощью библиотеки NetworkX.
    
    Parameters:
    G (nx.Graph): Граф для отображения.
    n (int): Значение n для n-грамм.
    
    Returns:
    nx.Graph: Отображенный граф.
    """
    
    plt.figure(figsize=(50, 50))
    # Определение позиций узлов для отображения
    pos = nx.spring_layout(G)  

    # Визуализация графа
    nx.draw(G, pos, with_labels=True, font_weight='bold', node_size=500, node_color='lightblue', font_size=10)

    # Добавление подписей с весами на рёбрах
    labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)

    plt.tight_layout()

    # Сохранение нарисованного графа в формате jpeg
    plt.savefig(f'{n}-grams.jpeg')
    plt.show()
    
    return G


In [ ]:
# Построение графа для 2-грамм
data_2 = count_word_occurrences_in_ngrams(create_ngrams(2, 100))
G2 = build_graph(data_2)
draw_graph(G2, 2)

In [ ]:
# Построение графа для 3-грамм
data_3 = count_word_occurrences_in_ngrams(create_ngrams(3, 100))
G3 = build_graph(data_3)
draw_graph(G3, 3)

In [ ]:
# Построение графа для 7-грамм
data_7 = count_word_occurrences_in_ngrams(create_ngrams(7, 100))
G7 = build_graph(data_7)
draw_graph(G7, 7)

# Построение вектора частот встречаемости его соседей

In [70]:
def get_vector(
        data: typing.Dict[str, typing.Dict[str, int]]) -> typing.Dict[str, list[int]]:
    """
    Строит вектор частот встречаемости соседий слова,
    заполняя нулями отсутствующие значения.

    Args:
        data (Dict[str, Dict[str, int]]): Словарь с данными о соседних словах 
        и их частоте встречаемости.

    Returns:
        Dict[str, list[int]: Словарь с векторами частот встречаемости слов.
    """
    # Длина словаря
    length = len(data)

    # Пустой словарь для векторов
    vectors = {}

    # Построение вектора частот встречаемости соседей слова
    for word, neighbors in tqdm(data.items()):
        vectors[word] = sorted(list(neighbors.values()), reverse=True)

        # Заполнение нулями отсутсвующие значения
        if len(vectors[word]) < length:
            while len(vectors[word]) != length:
                vectors[word].append(0)
    return vectors

In [71]:
from statistics import mean

In [72]:
def zipf(vectors: typing.Dict[str, list[int]]) -> list[str]:
    """
    Вычисляет коэффициент Ципфа для каждого слова и возвращает список слов,
    у которых коэффициент больше чем в два раза выше среднего.

    Args:
        vectors (Dict[str, list[int]]): Словарь с векторами частот встречаемости слов.

    Returns:
        list: Список слов, у которых коэффициент Ципфа выше чем в два раза среднего.
    """
    
    # Пустой словарь для показателя Ципфа для каждого слова словаря текста
    zipf_values = {}

    # Расчет показателя Ципфа как отношение частот первых двух самых частотных слов
    for word, vector in tqdm(vectors.items()):
        if vector[1] != 0:
            zipf_values[word] = vector[0] / vector[1]
        else:
            zipf_values[word] = 0
    
    # Подсчет среднего значения показателя Ципфа
    average = mean(list(zipf_values.values()))

    # Нахождение слов, чей показатель Ципфа 
    # выше более чем в 2 раза среднего значения
    high_than_average = [k for k, v in zipf_values.items() if v > 2 * average]

    return high_than_average

In [ ]:
vectors_2 = get_vector(data_2)
zipf_2 = zipf(vectors_2)
zipf_2

In [ ]:
vectors_3 = get_vector(data_3)
zipf_3 = zipf(vectors_3)
zipf_3

In [ ]:
vectors_7 = get_vector(data_7)
zipf_7 = zipf(vectors_7)
zipf_7

In [ ]:
# Поиск слов, у которых показатель Ципфа в два раза 
# больше среднего по ВСЕМ текстам (на примере 4-грамм)
all_data_4 = count_word_occurrences_in_ngrams(create_ngrams(4))
all_vectors_4 = get_vector(all_data_4)
zipf_4 = zipf(all_vectors_4 )
zipf_4

# Творческое задание

Определение степени схожести текстов, написанные одним автором, через подсчет косинусного расстояния между текстами.

Гипотеза: тексты одного автора должны быть похожи.

In [ ]:
%pip install scikit-learn
%pip install pandas
%pip install nltk

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from tqdm.auto import tqdm
from itertools import combinations
from statistics import mean

In [17]:
def vectorized_texts(author: str) -> typing.Tuple[typing.List[str], typing.List]:
    """
    Векторизует тексты из файла и возвращает названия статей и их векторное представление.

    Args:
        author (str): Имя автора, для которого нужно найти статьи.

    Returns:
        Tuple[List[str], List]: Названия статей и их векторное представление.
    """

    # Чтение текстов из файла и обработка
    with open('lemmatized_text.txt', 'r', encoding='utf-8') as file:
        texts = file.read().replace('[', '').replace(
            ']', '').replace("'", '').replace(', ', '').split('\n')[:-1]

    df = pd.read_csv('Doctor_Piter.csv')
    df['лемматизированный текст статьи'] = texts

    # Фильтрация датафрейма по имени автора
    author_articles = df[df['автор'] == author]
    
    # Получение списка названий статей и текстов статей
    titles = author_articles['название статьи'].tolist()
    texts = author_articles['лемматизированный текст статьи'].tolist()

    # Векторизация текстов
    tfidf = TfidfVectorizer()
    vectors = tfidf.fit_transform(texts)

    return titles, vectors


In [15]:
def find_similar_texts(author: str) -> typing.Tuple[typing.List[str], float]:
    """
    Находит похожие тексты на основе косинусного расстояния между векторами текстов.

    Args:
        author (str): Имя автора, для которого нужно найти статьи.
    Returns:
        Tuple[List[str], float]: Список с названиями самых похожих статей 
        и средним сходством статей автора.
    """

    # нахождение векторов текстов и их названий
    titles, vectors = vectorized_texts(author)

    # составление всех возможных пар текстов
    pairs = list(combinations(range(len(titles)), 2))
    pairs = [list(comb) for comb in pairs]

    # пустой список для похожих текстов
    similar_texts = []

    # определение похожести текстов
    for p in range(len(pairs)):
        f = pairs[p][0]
        s = pairs[p][1]
        similar_texts.append([titles[f], 
                              titles[s], 
                              cosine_similarity(vectors[f].toarray(),
                                                 vectors[s].toarray())[0][0]])

    # самые похожие статьи
    most_similar_texts = sorted(similar_texts, 
                                key=lambda x: x[2], 
                                reverse=True)[0]

    # средний уровень похожести 
    med = mean([s[2] for s in similar_texts])
    
    return most_similar_texts, med


In [5]:
# cписок авторов, у которых больше 40 статей
df = pd.read_csv('Doctor_Piter.csv')
df['автор'] = df['автор'].str.replace('ё', 'е')
authors = df.dropna(subset=['автор'])['автор'].value_counts()
authors = authors[authors > 40].index.tolist()

In [ ]:
authors

In [ ]:
# Нахождение показателя схожести текстов для каждого автора в списке
for author in tqdm(authors):
    most_similar_texts, med = find_similar_texts(author)

    # Сохранение данных
    with open('similarity_rate.txt', 'a', encoding='utf-8') as file:
        file.writelines(f'{author}\n')
        file.writelines(f'{str(med)}\n')
        file.writelines(f'Самые похожие статьи автора: "{most_similar_texts[0]}" и "{most_similar_texts[1]}"'\
                            f' (косинусное расстояние -- {most_similar_texts[2]}).\n')
        file.writelines('=====\n')

Вывод: тексты одного автора оказались непохожими, что объясняется:
- либо слишком разной тематикой текстов, 
- либо неверным способом определения близости текстов, 
- либо отсутсвием стиля у автора. 